# Olympic Medal Count

— Quentin Lehn and Lisa Pugh

```
Ever wonder which countries have earned the most Olympic medals ever?
Enter the type of Olympic games (summer/winter/both) and enter which type of medal (bronze/silver/gold)
to find out the top five countries with the most medals in that category.

Data source: https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table
             https://developers.google.com/public-data/docs/canonical/countries_csv
```


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import folium

def getHtml():
    url = "https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"
    response = requests.get(url)
    if response.ok:
        print(response)
    else:
        print("Error:", response.status_code)

getHtml()

<Response [200]>


# Grabbing the Data

```
We used two tables from two different webpages (Wikipedia and developers.google.com).
We extracted the title of the Wikipedia table using BeautifulSoup.
The first dataframe has the information about the Olympics and medal counts for the countries.
The second dataframe has the country codes and coordinates for each country.
We converted them to dataframes using Pandas, then merged them to form a new dataframe to work with from there.
```


In [2]:
url = "https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"
data = pd.read_html(url)

# Gets the title of the Wiki table using BeautifulSoup
def get_title():
    source_code = requests.get('https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table')
    soup = BeautifulSoup(source_code.content, "lxml")
    table_title = soup.select('#firstHeading')
    header = table_title[0].text
    return header

# Creates a Dataframe using Webscraping with the Wiki table data
def get_df():
    total_medals = pd.DataFrame()
    total_medals = total_medals.append(data[2], ignore_index=True)
    total_medals = total_medals.drop(total_medals.index[[0,1]])
    total_medals = total_medals.drop(total_medals.index[[-1]])
    del total_medals[1]
    del total_medals[6]
    del total_medals[11]
    total_medals.columns = ['Countries', 'Gold1', 'Silver1', 'Bronze1', 'Summer Total', 'Gold2', 'Silver2', 'Bronze2', 'Winter Total', 'Gold3','Silver3','Bronze3', 'Combined Total']
    return total_medals

# Formats the Dataframe and output it
# Makes a new column using the values
# from 'Countries' without the 3-letter Country Codes
header = "Dataframe #1: " + get_title()
print("{: ^125s}".format(header))
tm = get_df()
country_code_list=[]
for country in tm['Countries'].astype('str'):
    country_code = country.split("(")[0]
    country_code_list.append( country_code.strip() )
se = pd.Series(country_code_list)
tm['Country'] = se.values
tm

                                      Dataframe #1: All-time Olympic Games medal table                                       


,Countries,Gold1,Silver1,Bronze1,Summer Total,Gold2,Silver2,Bronze2,Winter Total,Gold3,Silver3,Bronze3,Combined Total,Country
2,Afghanistan (AFG),0,0,2,2,0,0,0,0,0,0,2,2,Afghanistan
3,Algeria (ALG),5,4,8,17,0,0,0,0,5,4,8,17,Algeria
4,Argentina (ARG),21,25,28,74,0,0,0,0,21,25,28,74,Argentina
5,Armenia (ARM),2,5,7,14,0,0,0,0,2,5,7,14,Armenia
6,Australasia (ANZ) [ANZ],3,4,5,12,0,0,0,0,3,4,5,12,Australasia
7,Australia (AUS) [AUS] [Z],147,163,187,497,5,3,4,12,152,166,191,509,Australia
8,Austria (AUT),18,33,36,87,59,78,81,218,77,111,117,305,Austria
9,Azerbaijan (AZE),7,11,25,43,0,0,0,0,7,11,25,43,Azerbaijan
10,Bahamas (BAH),6,2,6,14,0,0,0,0,6,2,6,14,Bahamas
11,Bahrain (BRN),1,1,1,3,0,0,0,0,1,1,1,3,Bahrain


In [3]:
url2 = "https://developers.google.com/public-data/docs/canonical/countries_csv"
data2 = pd.read_html(url2)

# Creates second dataframe from developers.google.com
def get_df2():
    coords_df = pd.DataFrame()
    coords_df = coords_df.append(data2[0], ignore_index=True)
    coords_df.sample()
    coords_df.columns = ['Code', 'Lat', 'Long', 'Country']
    coords_df = coords_df.drop(coords_df.index[[0,1]])
    return coords_df

# formats title and output first 10 elements of dataframe
print("{: ^50s}".format("Dataframe #2: Country Coordinates"))
cdf = get_df2()
cdf.head(10)

        Dataframe #2: Country Coordinates         


,Code,Lat,Long,Country
2,AE,23.424076,53.847818,United Arab Emirates
3,AF,33.93911,67.709953,Afghanistan
4,AG,17.060816,-61.796428,Antigua and Barbuda
5,AI,18.220554,-63.068615,Anguilla
6,AL,41.153332,20.168331,Albania
7,AM,40.069099,45.038189,Armenia
8,AN,12.226079,-69.060087,Netherlands Antilles
9,AO,-11.202692,17.873887,Angola
10,AQ,-75.250973,-0.071389,Antarctica
11,AR,-38.416097,-63.616672,Argentina


In [4]:
# Merges both dataframes, formats header, then outputs both
combined_df = pd.merge(left=tm, right=cdf, how='inner', left_on='Country', right_on='Country')
print("{: ^125s}".format("Combined Dataframes"))
combined_df

                                                     Combined Dataframes                                                     


,Countries,Gold1,Silver1,Bronze1,Summer Total,Gold2,Silver2,Bronze2,Winter Total,Gold3,Silver3,Bronze3,Combined Total,Country,Code,Lat,Long
0,Afghanistan (AFG),0,0,2,2,0,0,0,0,0,0,2,2,Afghanistan,AF,33.93911,67.709953
1,Algeria (ALG),5,4,8,17,0,0,0,0,5,4,8,17,Algeria,DZ,28.033886,1.659626
2,Argentina (ARG),21,25,28,74,0,0,0,0,21,25,28,74,Argentina,AR,-38.416097,-63.616672
3,Armenia (ARM),2,5,7,14,0,0,0,0,2,5,7,14,Armenia,AM,40.069099,45.038189
4,Australia (AUS) [AUS] [Z],147,163,187,497,5,3,4,12,152,166,191,509,Australia,AU,-25.274398,133.775136
5,Austria (AUT),18,33,36,87,59,78,81,218,77,111,117,305,Austria,AT,47.516231,14.550072
6,Azerbaijan (AZE),7,11,25,43,0,0,0,0,7,11,25,43,Azerbaijan,AZ,40.143105,47.576927
7,Bahamas (BAH),6,2,6,14,0,0,0,0,6,2,6,14,Bahamas,BS,25.03428,-77.39628
8,Bahrain (BRN),1,1,1,3,0,0,0,0,1,1,1,3,Bahrain,BH,25.930414,50.637772
9,Barbados (BAR) [BAR],0,0,1,1,0,0,0,0,0,0,1,1,Barbados,BB,13.193887,-59.543198


# The Medal Counter

```
Now to put that data to use...

Input the type of Olympics you want the medal count of: summer, winter, or both combined.
Then, input the type of medal you want to see the medal count for.
The output will be the top 5 countries in order with the most medals for the specified input.
Try it again until you exit the program (quit).
```

In [8]:
#Quit Function
def quit():
    print("Exiting All-Time Olympic Medal Counter...")

#Main Program
print("All-Time Olympic Medal Counter")

while True:
    combined_df = pd.merge(left=tm, right=cdf, how='inner', left_on='Country', right_on='Country')
    print('=' * 55)
    season_type = input("Please enter (summer/winter/both) or quit: ")
    if season_type.lower() == 'quit':
        quit()
        break
    elif season_type.lower() == 'summer':
        medal_type = input("Please enter (bronze/silver/gold) or quit: ")
        print("")
        if medal_type.lower() == 'quit':
            quit()
            break
        elif medal_type.lower() == 'bronze':
            del combined_df['Countries'], combined_df['Silver1'], combined_df['Gold1'], combined_df['Summer Total'], combined_df['Bronze2'], combined_df['Silver2'], combined_df['Gold2'], combined_df['Winter Total'], combined_df['Gold3'], combined_df['Silver3'], combined_df['Bronze3'], combined_df['Combined Total'], combined_df['Lat'], combined_df['Long']
            combined_df['Bronze1'] = combined_df['Bronze1'].astype('int')
            sb = combined_df.sort_values(['Bronze1'], ascending=False).head(5)
            print('All-Time Bronze Medals Awarded in Summer Olympics')
            print(sb.to_string(index=False, header=False))
        elif medal_type.lower() == 'silver':
            del combined_df['Countries'], combined_df['Bronze1'], combined_df['Gold1'], combined_df['Summer Total'], combined_df['Bronze2'], combined_df['Silver2'], combined_df['Gold2'], combined_df['Winter Total'], combined_df['Gold3'], combined_df['Silver3'], combined_df['Bronze3'], combined_df['Combined Total'], combined_df['Lat'], combined_df['Long']
            combined_df['Silver1'] = combined_df['Silver1'].astype('int')
            ss = combined_df.sort_values(['Silver1'], ascending=False).head(5)
            print('All-Time Silver Medals Awarded in Summer Olympics')
            print(ss.to_string(index=False, header=False))
        elif medal_type.lower() == 'gold':
            del combined_df['Countries'], combined_df['Bronze1'], combined_df['Silver1'], combined_df['Summer Total'], combined_df['Bronze2'], combined_df['Silver2'], combined_df['Gold2'], combined_df['Winter Total'], combined_df['Gold3'], combined_df['Silver3'], combined_df['Bronze3'], combined_df['Combined Total'], combined_df['Lat'], combined_df['Long']
            combined_df['Gold1'] = combined_df['Gold1'].astype('int')
            sg = combined_df.sort_values(['Gold1'], ascending=False).head(5)
            print('All-Time Gold Medals Awarded in Summer Olympics')
            print(sg.to_string(index=False, header=False))
        else:
            print('ERROR:', medal_type,'is not a medal type.')
    elif season_type.lower() == 'winter':
        medal_type = input("Please enter (bronze/silver/gold) or quit: ")
        print("")
        if medal_type.lower() == 'quit':
            quit()
            break
        elif medal_type.lower() == 'bronze':
            del combined_df['Countries'], combined_df['Bronze1'], combined_df['Silver1'], combined_df['Gold1'], combined_df['Summer Total'], combined_df['Silver2'], combined_df['Gold2'], combined_df['Winter Total'], combined_df['Gold3'], combined_df['Silver3'], combined_df['Bronze3'], combined_df['Combined Total'], combined_df['Lat'], combined_df['Long']
            combined_df['Bronze2'] = combined_df['Bronze2'].astype('int')
            wb = combined_df.sort_values(['Bronze2'], ascending=False).head(5)
            print('All-Time Bronze Medals Awarded in Winter Olympics')
            print(wb.to_string(index=False, header=False))            
        elif medal_type.lower() == 'silver':
            del combined_df['Countries'], combined_df['Bronze1'], combined_df['Silver1'], combined_df['Gold1'], combined_df['Summer Total'], combined_df['Bronze2'], combined_df['Gold2'], combined_df['Winter Total'], combined_df['Gold3'], combined_df['Silver3'], combined_df['Bronze3'], combined_df['Combined Total'], combined_df['Lat'], combined_df['Long']
            combined_df['Silver2'] = combined_df['Silver2'].astype('int')
            ws = combined_df.sort_values(['Silver2'], ascending=False).head(5)
            print('All-Time Silver Medals Awarded in Winter Olympics')
            print(ws.to_string(index=False, header=False)) 
        elif medal_type.lower() == 'gold':
            del combined_df['Countries'], combined_df['Bronze1'], combined_df['Silver1'], combined_df['Gold1'], combined_df['Summer Total'], combined_df['Bronze2'], combined_df['Silver2'], combined_df['Winter Total'], combined_df['Gold3'], combined_df['Silver3'], combined_df['Bronze3'], combined_df['Combined Total'], combined_df['Lat'], combined_df['Long']
            combined_df['Gold2'] = combined_df['Gold2'].astype('int')
            wg = combined_df.sort_values(['Gold2'], ascending=False).head(5)
            print('All-Time Gold Medals Awarded in Winter Olympics')
            print(wg.to_string(index=False, header=False)) 
        else:
            print('ERROR:', medal_type,'is not a medal type.')
    elif season_type.lower() == 'both':
        medal_type = input("Please enter (bronze/silver/gold) or quit: ")
        print("")
        if medal_type.lower() == 'quit':
            quit()
            break
        elif medal_type.lower() == 'bronze':
            del combined_df['Countries'], combined_df['Bronze1'], combined_df['Silver1'], combined_df['Gold1'], combined_df['Summer Total'], combined_df['Bronze2'], combined_df['Silver2'], combined_df['Gold2'], combined_df['Winter Total'], combined_df['Gold3'], combined_df['Silver3'], combined_df['Combined Total'], combined_df['Lat'], combined_df['Long']
            combined_df['Bronze3'] = combined_df['Bronze3'].astype('int')
            bb = combined_df.sort_values(['Bronze3'], ascending=False).head(5)
            print('All-Time Bronze Medals Awarded in the Olympics')
            print(bb.to_string(index=False, header=False))
        elif medal_type.lower() == 'silver':
            del combined_df['Countries'], combined_df['Bronze1'], combined_df['Silver1'], combined_df['Gold1'], combined_df['Summer Total'], combined_df['Bronze2'], combined_df['Silver2'], combined_df['Gold2'], combined_df['Winter Total'], combined_df['Gold3'], combined_df['Bronze3'], combined_df['Combined Total'], combined_df['Lat'], combined_df['Long']
            combined_df['Silver3'] = combined_df['Silver3'].astype('int')
            bs = combined_df.sort_values(['Silver3'], ascending=False).head(5)
            print('All-Time Silver Medals Awarded in the Olympics')
            print(bs.to_string(index=False, header=False))
        elif medal_type.lower() == 'gold':
            del combined_df['Countries'], combined_df['Bronze1'], combined_df['Silver1'], combined_df['Gold1'], combined_df['Summer Total'], combined_df['Bronze2'], combined_df['Silver2'], combined_df['Gold2'], combined_df['Winter Total'], combined_df['Bronze3'], combined_df['Silver3'], combined_df['Combined Total'], combined_df['Lat'], combined_df['Long']
            combined_df['Gold3'] = combined_df['Gold3'].astype('int')
            bg = combined_df.sort_values(['Gold3'], ascending=False).head(5)
            print('All-Time Gold Medals Awarded in the Olympics')
            print(bg.to_string(index=False, header=False))
        else:
            print('ERROR:', medal_type,'is not a medal type.')
    else:
        print('ERROR:', season_type,'is not a season type.')

All-Time Olympic Medal Counter
Please enter (summer/winter/both) or quit: summer
Please enter (bronze/silver/gold) or quit: gold

All-Time Gold Medals Awarded in Summer Olympics
1022  United States  US
 224          China  CN
 212         France  FR
 206          Italy  IT
 191        Germany  DE
Please enter (summer/winter/both) or quit: winter
Please enter (bronze/silver/gold) or quit: silver

All-Time Silver Medals Awarded in Winter Olympics
111         Norway  NO
102  United States  US
 78        Germany  DE
 78        Austria  AT
 62        Finland  FI
Please enter (summer/winter/both) or quit: both
Please enter (bronze/silver/gold) or quit: gold

All-Time Gold Medals Awarded in the Olympics
1118  United States  US
 269        Germany  DE
 243          Italy  IT
 243         France  FR
 236          China  CN
Please enter (summer/winter/both) or quit: both gold
ERROR: both gold is not a season type.
Please enter (summer/winter/both) or quit: fdjfhdkhfdksjhfkds
ERROR: fdjfhdkhfdksj

# Map of Top 5 Countries with Most Olympic Gold Medals Ever

```
We can do even more with the data we gathered...
such as mapping it using Folium.

Here is a map displaying the top 5 countries at their corresponding geolocations
that have won the most medals in both the summer and winter olympics combined.
```

In [9]:
# Sorts 'Combined Total' column as integers
# Outputs it without row index or column titles
combined_df = pd.merge(left=tm, right=cdf, how='inner', left_on='Country', right_on='Country')
del combined_df['Countries'], combined_df['Bronze1'], combined_df['Silver1'], combined_df['Gold1'], combined_df['Summer Total'], combined_df['Bronze2'], combined_df['Silver2'], combined_df['Gold2'], combined_df['Winter Total'], combined_df['Gold3'], combined_df['Silver3'], combined_df['Bronze3']
combined_df['Combined Total'] = combined_df['Combined Total'].astype('int')
sorted_total = combined_df.sort_values(['Combined Total'], ascending=False).head(5)
print('All-Time Medals Awarded in the Olympics')
print(sorted_total.to_string(index=False, header=False))

# Creates Folium Map of World with 5 values from 'Combined Total'
CENTER_WORLD = (0,0)
country_geojson = 'world-countries.json'
map = folium.Map(location=CENTER_WORLD, zoom_start=2, tiles='Open Street Map')
for row in sorted_total.sample(5).to_records():
    pos = (row['Lat'],row['Long'])
    marker = folium.Marker(location=pos, 
                    popup="Medal Count: %s, %s (%s)" % (row['Combined Total'], row['Country'], row['Code'])
                          )
    map.add_child(marker)
map

All-Time Medals Awarded in the Olympics
2803  United States  US   37.09024  -95.712891
 824        Germany  DE  51.165691   10.451526
 824         France  FR  46.227638    2.213749
 691          Italy  IT   41.87194    12.56738
 638         Sweden  SE  60.128161   18.643501
